Set Up Your Environment

In [1]:
%pip install pandas numpy scikit-learn matplotlib


Note: you may need to restart the kernel to use updated packages.


# Código em Jupyter para Gráficos de Radar e Barras

In [10]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Carregando o arquivo Excel
file_path = r'C:\Users\Elaine\Downloads\inova_drones\Survey_cangerana\Survey_DRONES_14_01_25_CORONEL_CANGERANA.xlsx'

# Tentando ler o arquivo pulando as primeiras linhas que parecem conter metadados
df = pd.read_excel(file_path, engine='openpyxl', skiprows=5)

# Mostrando as primeiras linhas e colunas
print("Primeiras linhas do dataframe:")
print(df.head())
print("\nColunas disponíveis:")
print(df.columns)

# Removendo apenas linhas completamente vazias
df_cleaned = df.dropna(how='all')

# Mostrando os tipos de dados para identificar colunas numéricas
print("\nTipos de dados:")
print(df_cleaned.dtypes)

# Selecionando apenas colunas numéricas
df_numericas = df_cleaned.select_dtypes(include=[np.number])
print("\nColunas numéricas disponíveis:")
print(df_numericas.columns)

# Verificando se temos colunas suficientes para modelagem
if df_numericas.shape[1] >= 4:
    # Pegando as 3 primeiras como features e a 4ª como target (você pode ajustar conforme necessário)
    feature_columns = df_numericas.columns[:3].tolist()
    target_column = df_numericas.columns[3]

    X = df_numericas[feature_columns]
    y = df_numericas[target_column]

    # Dividindo os dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Treinando o modelo
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Fazendo previsões
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'\nMean Squared Error: {mse:.2f}')

    # Gráfico de barra
    plt.figure(figsize=(10, 6))
    sns.barplot(x=np.arange(len(y_test)), y=y_test.values, color='blue', label='Valores Reais')
    sns.barplot(x=np.arange(len(y_pred)), y=y_pred, color='red', alpha=0.6, label='Valores Preditos')
    plt.xlabel('Índice')
    plt.ylabel('Valores')
    plt.title('Valores Reais vs. Valores Preditos')
    plt.legend()
    plt.show()

    # Gráfico de radar com a média das features
    labels = np.array(feature_columns)
    stats = X.mean(axis=0).values
    stats = np.concatenate((stats, [stats[0]]))  # Fecha o gráfico

    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, stats, color='red', alpha=0.25)
    ax.plot(angles, stats, color='red', linewidth=2)

    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)

    plt.title('Gráfico de Radar das Features')
    plt.show()
else:
    print("\nNão há colunas numéricas suficientes para fazer a modelagem.")


Primeiras linhas do dataframe:
                             Objetivo do Site Survey  \
0            Qual é a finalidade do uso dos drones?    
1  Quais são os dados que precisam ser coletados ...   
2                            Localização geográfica:   
3                                    Área do projeto   
4                                    Tipo de terreno   

                                          Ambiente\n  Virtual\n(SIM / NÃO)  \
0  monitoramento ambiental, coleta de dados geoes...                   NaN   
1                                                NaN                   NaN   
2         Latitude e longitude do local de operação.                   NaN   
3  Qual é o tamanho da área que será coberta pelo...                   NaN   
4          Urbano, rural, montanhoso, aquático, etc.                   NaN   

   Versão / Detalhamento   Obs  
0                     NaN  NaN  
1                     NaN  NaN  
2                     NaN  NaN  
3                     NaN  NaN 

# Exemplo de código que analisa e plota

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

# Garante a exibição dos gráficos no Jupyter Notebook
%matplotlib inline

# Carregando o arquivo
file_path = r'C:\Users\Elaine\Downloads\inova_drones\Survey_cangerana\Survey_DRONES_14_01_25_CORONEL_CANGERANA.xlsx'
df = pd.read_excel(file_path, engine='openpyxl', skiprows=5)

# Limpando espaços e quebras de linha nos nomes das colunas
df.columns = df.columns.str.strip().str.replace('\n', '', regex=False)

# Verificando colunas disponíveis
print("Colunas disponíveis:", df.columns.tolist())

# Gráfico 1: Contagem da coluna "Virtual (SIM / NÃO)"
col_virtual = 'Virtual(SIM / NÃO)'  # Coluna já tratada acima

if col_virtual in df.columns:
    plt.figure(figsize=(6, 4))
    sns.countplot(x=df[col_virtual].dropna(), palette='Set2')
    plt.title('Distribuição de respostas: Virtual (SIM / NÃO)')
    plt.xlabel('Resposta')
    plt.ylabel('Contagem')
    plt.tight_layout()
    plt.savefig("grafico_virtual.png")  # Salva o gráfico como imagem
    plt.show()
else:
    print(f"⚠️ Coluna '{col_virtual}' não encontrada.")

# Gráfico 2: Nuvem de palavras da coluna "Objetivo do Site Survey"
col_objetivo = 'Objetivo do Site Survey'

if col_objetivo in df.columns:
    texto = ' '.join(str(resposta) for resposta in df[col_objetivo].dropna())
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(width=800, height=400, background_color='white',
                          stopwords=stopwords, colormap='viridis').generate(texto)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Nuvem de Palavras: Objetivo do Site Survey')
    plt.tight_layout()
    wordcloud.to_file("nuvem_palavras.png")  # Salva como imagem
    plt.show()
else:
    print(f"⚠️ Coluna '{col_objetivo}' não encontrada.")


In [ ]:
%pip install wordcloud

   ---------------------------------------- 0.0/299.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/299.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/299.9 kB ? eta -:--:--
   --- ----------------------------------- 30.7/299.9 kB 217.9 kB/s eta 0:00:02
   ------- ------------------------------- 61.4/299.9 kB 363.1 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/299.9 kB 438.1 kB/s eta 0:00:01
   ------------------- ------------------ 153.6/299.9 kB 612.6 kB/s eta 0:00:01
   ------------------------- ------------ 204.8/299.9 kB 655.1 kB/s eta 0:00:01
   ------------------------------------ - 286.7/299.9 kB 803.7 kB/s eta 0:00:01
   -------------------------------------- 299.9/299.9 kB 772.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
from wordcloud import WordCloud, STOPWORDS


In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

# Garante a exibição dos gráficos no Jupyter Notebook
%matplotlib inline

# Carregando o arquivo
file_path = r'C:\Users\Elaine\Downloads\inova_drones\Survey_cangerana\Survey_DRONES_14_01_25_CORONEL_CANGERANA.xlsx'
df = pd.read_excel(file_path, engine='openpyxl', skiprows=5)

# Limpando espaços e quebras de linha nos nomes das colunas
df.columns = df.columns.str.strip().str.replace('\n', '', regex=False)

# Verificando colunas disponíveis
print("Colunas disponíveis:", df.columns.tolist())

# Gráfico 1: Contagem da coluna "Virtual (SIM / NÃO)"
col_virtual = 'Virtual(SIM / NÃO)'  # Coluna já tratada acima

if col_virtual in df.columns:
    plt.figure(figsize=(6, 4))
    sns.countplot(x=df[col_virtual].dropna(), palette='Set2')
    plt.title('Distribuição de respostas: Virtual (SIM / NÃO)')
    plt.xlabel('Resposta')
    plt.ylabel('Contagem')
    plt.tight_layout()
    plt.savefig("grafico_virtual.png")  # Salva o gráfico como imagem
    plt.show()
else:
    print(f"⚠️ Coluna '{col_virtual}' não encontrada.")

# Gráfico 2: Nuvem de palavras da coluna "Objetivo do Site Survey"
col_objetivo = 'Objetivo do Site Survey'

if col_objetivo in df.columns:
    texto = ' '.join(str(resposta) for resposta in df[col_objetivo].dropna())
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(width=800, height=400, background_color='white',
                          stopwords=stopwords, colormap='viridis').generate(texto)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Nuvem de Palavras: Objetivo do Site Survey')
    plt.tight_layout()
    wordcloud.to_file("nuvem_palavras.png")  # Salva como imagem
    plt.show()
else:
    print(f"⚠️ Coluna '{col_objetivo}' não encontrada.")

Colunas disponíveis: ['Objetivo do Site Survey', 'Ambiente', 'Virtual(SIM / NÃO)', 'Versão / Detalhamento', 'Obs']


ValueError: min() arg is an empty sequence

<Figure size 600x400 with 0 Axes>

In [20]:
col_virtual = 'Virtual(SIM / NÃO)'

if col_virtual in df.columns:
    dados_validos = df[col_virtual].dropna()
    if not dados_validos.empty:
        plt.figure(figsize=(6, 4))
        sns.countplot(x=dados_validos, palette='Set2')
        plt.title('Distribuição de respostas: Virtual (SIM / NÃO)')
        plt.xlabel('Resposta')
        plt.ylabel('Contagem')
        plt.tight_layout()
        plt.savefig("grafico_virtual.png")
        plt.show()
    else:
        print(f"⚠️ A coluna '{col_virtual}' está vazia ou só contém valores nulos.")
else:
    print(f"⚠️ Coluna '{col_virtual}' não encontrada no DataFrame.")


⚠️ A coluna 'Virtual(SIM / NÃO)' está vazia ou só contém valores nulos.


# A planilha (Survey_DRONES_14_01_25_CORONEL_CANGERANA.xlsx) esta base de dados possui dados nulos que não podem ser tratados ou correlaciona-los